# Prepare Power Flow Data

This notebook presents the process of data preparation for building power flow regimes. The process is separated into stages (see, [DVC config](../dvc.yaml)):
 - "parse" --- extract necessary parameters from the raw data
 - "transform" --- combine and convert data to use in further steps
 - "prepare" --- build final dataset

The naming of variables in the final dataset corresponds to the [project convention](../convention.md).

Since the NREL-118 dataset is mostly intended for OPF tasks, it skips a lot of information that is not necessary to solve these kind of tasks. Thus, to append the data with missing info, [JEAS-118 dataset](http://motor.ece.iit.edu/data/JEAS_IEEE118.doc) is used as the primary source of NREL-118. The following sections describe all the stages of data processing and all the decisions made when preparing the final dataset.

In [1]:
import os

from src.data import parse_jeas118_lines
from src.data import parse_jeas118_loads
from src.data import parse_jeas118_trafos
from src.data import parse_nrel118_buses
from src.data import parse_nrel118_escalators_ts
from src.data import parse_nrel118_gens
from src.data import parse_nrel118_hydros_nondisp_ts
from src.data import parse_nrel118_hydros_ts
from src.data import parse_nrel118_lines
from src.data import parse_nrel118_loads_ts
from src.data import parse_nrel118_outages_ts
from src.data import parse_nrel118_solars_ts
from src.data import parse_nrel118_winds_ts
from src.data import prepare_branches
from src.data import prepare_buses
from src.data import prepare_gens
from src.data import prepare_gens_ts
from src.data import prepare_loads
from src.data import prepare_loads_ts
from src.data import transform_gens_escalated_ts
from src.data import transform_loads


PATH_NREL118 = os.path.join("..", "data", "raw", "nrel118")
PATH_JEAS118 = os.path.join("..", "data", "raw", "jeas118")

## Buses

To build a power flow model, the following information about buses is necessary:
- rated voltage level
- if the bus is in service or out of service
- name (optional)
- region (optional)

Let's load and parse bus data of NREL-118 power system:

In [2]:
path_nrel118_buses = os.path.join(PATH_NREL118, "additional-files-mti-118", "Buses.csv")
nrel118_buses = parse_nrel118_buses(raw_data=path_nrel118_buses)
nrel118_buses.head(2)

,bus_name,region,load_participation_factor
0,bus__1,r1,0.047169
1,bus__2,r1,0.018496


The NREL-118 dataset contains only names and regions of buses ("load_participation_factor" is for load modelling, see [Section "Loads"](#loads)). To add missing values, it is assumed the following:
- all buses are in service
- rated voltage level of buses 8, 9, 10, 26, 30, 38, 63, 64, 65, 68, 81, 116 equals to 345 kV, the rest of buses has the voltage level of 138 kV. This corresponds to transformers data from JEAS-118 dataset.

Thus, the final bus data look as follows:

In [3]:
buses = prepare_buses(parsed_nrel118_buses=nrel118_buses)
buses.head(2)

,bus_name,region,in_service,v_rated__kv
0,bus__1,r1,True,138
1,bus__2,r1,True,138


## Branches

"Branches" is a common term both for lines and transformers. The following parameters about branches are necessary to build models:
- start and end buses of the branch
- number of parallel branch systems
- resistance
- reactance
- active conductance
- in service or out of service
- maximum power flow (optional)
- transformation ratio if the branch is a transformer (optional)
- name (optional)

Let's load and parse line data of NREL-118 power system:

In [4]:
path_nrel118_lines = os.path.join(PATH_NREL118, "additional-files-mti-118", "Lines.csv")
nrel118_lines = parse_nrel118_lines(raw_data=path_nrel118_lines)
nrel118_lines.head(2)

,branch_name,from_bus,to_bus,max_p__mw,x__pu,r__pu
0,branch__1,bus__1,bus__2,600.0,0.0999,0.0303
1,branch__2,bus__1,bus__3,600.0,0.0424,0.0129


Since the information about active conductance and parallel number is skipped in the NREL-118 dataset, let's load it from the JEAS-118 dataset:

In [5]:
path_jeas118_lines = os.path.join(PATH_JEAS118, "JEAS_IEEE118.doc")
jeas118_lines = parse_jeas118_lines(raw_data=path_jeas118_lines)
jeas118_lines.head(2)

,branch_name,from_bus,to_bus,parallel,b__pu
0,branch__1,bus__1,bus__2,1,0.02540
1,branch__2,bus__1,bus__3,1,0.01082


In the NREL-118 dataset, transformers are presented as lines without values of transformation ratio. Therefore, these values will be loaded from JEAS-118 dataset:

In [6]:
path_jeas118_trafos = os.path.join(PATH_JEAS118, "JEAS_IEEE118.doc")
jeas118_trafos = parse_jeas118_trafos(raw_data=path_jeas118_trafos)
jeas118_trafos.head(2)

,branch_name,from_bus,to_bus,parallel,trafo_ratio
0,trafo__1,bus__8,bus__5,1,0.985
1,trafo__2,bus__26,bus__25,1,0.960


Thus, the final branch data look as follows:

In [7]:
branches = prepare_branches(
    parsed_nrel118_lines=nrel118_lines,
    parsed_jeas118_lines=jeas118_lines,
    parsed_jeas118_trafos=jeas118_trafos,
    prepared_buses=buses,
)
branches.head(2)

,branch_name,from_bus,to_bus,parallel,in_service,r__ohm,x__ohm,b__µs,trafo_ratio,max_i__ka
0,branch__1,bus__1,bus__2,1,True,5.770332,19.024956,133.375341,NaN,2.510219
1,branch__2,bus__1,bus__3,1,True,2.456676,8.074656,56.815795,NaN,2.510219


## Loads

Here is the list of necessary load variables:

- bus where the load is located
- active and reactive power of the load
- if the load is in service
- name (optional)

The information about a part of the regional active load located in each bus is stored in variable "load_participation_factor" in the bus data of the NREL-118 dataset:

In [8]:
nrel118_buses.head(2)

,bus_name,region,load_participation_factor
0,bus__1,r1,0.047169
1,bus__2,r1,0.018496


Active load value of regions is stored in the time-series NREL-118 data:

In [9]:
path_nrel118_loads_ts = os.path.join(PATH_NREL118, "Input files", "RT", "Load")
nrel118_loads_ts = parse_nrel118_loads_ts(raw_data=path_nrel118_loads_ts)
nrel118_loads_ts.head(2)

,datetime,region_name,region_load
0,2024-01-01 00:00:00,r3,2421.205928
1,2024-01-01 00:00:00,r1,5698.083154


To calculate reactive power of loads, let's get the JEAS-118 data:

In [10]:
path_jeas118_loads = os.path.join(PATH_JEAS118, "JEAS_IEEE118.doc")
jeas118_loads = parse_jeas118_loads(raw_data=path_jeas118_loads)
jeas118_loads.head(2)

,bus_name,p__mw,q__mvar
0,bus__1,54.14,8.66
1,bus__2,21.23,9.55


The JEAS-118 load data will help to estimate the power factor of each load and define its reactive power at each moment of time using time-series data of active demand:

In [11]:
transformed_loads = transform_loads(
    parsed_nrel118_buses=nrel118_buses, parsed_jeas118_loads=jeas118_loads
)
transformed_loads.head(2)

,load_name,bus_name,region,load_participation_factor,load_power_factor
0,load__1,bus__1,r1,0.047169,0.987447
1,load__2,bus__2,r1,0.018496,0.911978


Thus, it is necessary to prepare two files with load data. The first will contain the load power variation over time, the other will contain basic load information (location, etc.).

In [12]:
loads = prepare_loads(transformed_loads=transformed_loads)
loads.head(2)

,load_name,bus_name
0,load__1,bus__1
1,load__2,bus__2


In [13]:
loads_ts = prepare_loads_ts(
    transformed_loads=transformed_loads, parsed_nrel118_loads_ts=nrel118_loads_ts
)
loads_ts.head(2)

,load__1__in_service,load__10__in_service,load__11__in_service,load__12__in_service,load__13__in_service,load__14__in_service,load__15__in_service,load__16__in_service,load__17__in_service,load__18__in_service,...,load__83__q__mvar,load__84__q__mvar,load__85__q__mvar,load__86__q__mvar,load__87__q__mvar,load__88__q__mvar,load__89__q__mvar,load__9__q__mvar,load__90__q__mvar,load__91__q__mvar
datetime,,,,,,,,,,,,,,,,,,,,,
2024-01-01 00:00:00,True,True,True,True,True,True,True,True,True,True,...,38.551556,3.255938,9.767815,97.678165,42.327208,15.786699,36.885267,84.344645,42.147499,16.031958
2024-01-01 01:00:00,True,True,True,True,True,True,True,True,True,True,...,39.589353,3.343587,10.030762,100.307633,43.466644,15.142957,35.381178,80.905282,40.42883,15.518813


## Generators


To build a power flow model, the following information about generators is necessary:
- bus where the generator is located
- active power of the generator
- if the generator is in service
- voltage set point of the generator
- max and min limits of reactive power output
- name (optional)

Let's start from parsing generator data from the NREL-118 dataset:

In [14]:
path_nrel118_gens = os.path.join(
    PATH_NREL118, "additional-files-mti-118", "Generators.csv"
)
nrel118_gens = parse_nrel118_gens(raw_data=path_nrel118_gens)
nrel118_gens.head(2)

,gen_name,bus_name,max_p__mw
0,biomass__1,bus__12,3.0
1,biomass__2,bus__12,3.0


Next, time-series data from the NREL-118 dataset are parsed:

In [15]:
# Hydro plants
path_nrel118_hydros_ts = os.path.join(PATH_NREL118, "Input files", "Hydro")
nrel118_hydros_ts = parse_nrel118_hydros_ts(raw_data=path_nrel118_hydros_ts)
nrel118_hydros_ts.head(2)

,datetime,gen_name,p__mw
0,2024-01-01 00:00:00,hydro__43,4.106700
1,2024-01-01 00:00:00,hydro__41,2.425893


In [16]:
# Solar plants
path_nrel118_solars_ts = os.path.join(PATH_NREL118, "Input files", "RT", "Solar")
nrel118_solars_ts = parse_nrel118_solars_ts(raw_data=path_nrel118_solars_ts)
nrel118_solars_ts.head(2)

,datetime,gen_name,p__mw
0,2024-01-01 00:00:00,solar__2,0.0
1,2024-01-01 00:00:00,solar__67,0.0


In [17]:
# Wind plants
path_nrel118_winds_ts = os.path.join(PATH_NREL118, "Input files", "RT", "Wind")
nrel118_winds_ts = parse_nrel118_winds_ts(raw_data=path_nrel118_winds_ts)
nrel118_winds_ts.head(2)

,datetime,gen_name,p__mw
0,2024-01-01 00:00:00,wind__10,12.765404
1,2024-01-01 00:00:00,wind__5,0.902116


In [18]:
# Non-dispatchable hydro plants
path_nrel118_hydros_nondisp_ts = os.path.join(
    PATH_NREL118,
    "additional-files-mti-118",
    "Hydro_nondipatchable.csv",
)
nrel118_hydros_nondisp_ts = parse_nrel118_hydros_nondisp_ts(
    raw_data=path_nrel118_hydros_nondisp_ts
)
nrel118_hydros_nondisp_ts.head(2)

,datetime,gen_name,p__mw
0,2024-01-01 00:00:00,hydro__36,0.51
1,2024-10-01 00:00:00,hydro__36,0.43


Escalators used to adjust generation profile to seasons or other time for all the generators, except wind, solar and hydro:

In [19]:
# Escalators data
path_nrel118_escalators_ts = os.path.join(
    PATH_NREL118, "additional-files-mti-118", "Escalators.csv"
)
nrel118_escalators_ts = parse_nrel118_escalators_ts(raw_data=path_nrel118_escalators_ts)
nrel118_escalators_ts.head(2)

,datetime,gen_name,escalator_ratio
0,2024-01-01 00:00:00,biomass__1,0.3500
1,2024-10-01 00:00:00,biomass__1,0.3433


Thus, it is possible to multiply max output of generation by escalators to estimate the power output over time of all generators, except wind, solar and hydro:

In [20]:
gens_escalated_ts = transform_gens_escalated_ts(
    parsed_nrel118_gens=nrel118_gens, parsed_nrel118_escalators_ts=nrel118_escalators_ts
)
gens_escalated_ts.head(2)

,datetime,gen_name,p__mw
0,2024-01-01 00:00:00,biomass__1,1.0500
1,2024-10-01 00:00:00,biomass__1,1.0299


In [21]:
# Outages
path_nrel118_outages_ts = os.path.join(
    PATH_NREL118, "Input files", "Others", "GenOut.csv"
)
nrel118_outages_ts = parse_nrel118_outages_ts(raw_data=path_nrel118_outages_ts)
nrel118_outages_ts.head(2)

,datetime,gen_name,in_outage
0,2024-01-01 00:00:00,biomass__1,False
1,2024-01-14 15:00:00,biomass__1,True


Concat all datasets to build two files with generation data --- general generation info (location, etc.), time-series data (p__mw, in_service, etc.).

In [22]:
prepared_gens = prepare_gens(parsed_nrel118_gens=nrel118_gens)
prepared_gens.head(2)

,gen_name,bus_name
0,biomass__1,bus__12
1,biomass__2,bus__12


In [23]:
prepared_gens_ts = prepare_gens_ts(
    parsed_nrel118_winds_ts=nrel118_winds_ts,
    parsed_nrel118_solars_ts=nrel118_solars_ts,
    parsed_nrel118_hydros_ts=nrel118_hydros_ts,
    parsed_nrel118_hydros_nondisp_ts=nrel118_hydros_nondisp_ts,
    transformed_gens_escalated_ts=gens_escalated_ts,
    parsed_nrel118_outages_ts=nrel118_outages_ts,
)
prepared_gens_ts.head(2)

,biomass__1__p__mw,biomass__10__p__mw,biomass__11__p__mw,biomass__12__p__mw,biomass__13__p__mw,biomass__14__p__mw,biomass__15__p__mw,biomass__16__p__mw,biomass__17__p__mw,biomass__18__p__mw,...,in_service__in_service,in_service__in_service,in_service__in_service,in_service__in_service,in_service__in_service,in_service__in_service,in_service__in_service,in_service__in_service,in_service__in_service,in_service__in_service
datetime,,,,,,,,,,,,,,,,,,,,,
2024-01-01 00:00:00,1.05,0.78003,0.379992,0.379992,0.379992,0.83995,0.200005,0.200005,0.200005,0.200005,...,True,True,True,True,True,True,True,True,True,True
2024-01-01 01:00:00,1.05,0.78003,0.379992,0.379992,0.379992,0.83995,0.200005,0.200005,0.200005,0.200005,...,True,True,True,True,True,True,True,True,True,True
